In [37]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv("dirty_cafe_sales.csv")

# 1. Asssesing the quality of the data
   Checking for:<br>
       -Missing values<br>
       -Duplicates<br>
       -Number of Rows and Columns
    

In [41]:
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [43]:
#Total nul values
df.isnull().sum()

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

In [45]:
# Incorrect data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [51]:
df.duplicated().sum()

0

In [53]:
df.shape

(10000, 8)

# 2. Data type standardization

In [55]:
df = df.replace(["ERROR", "UNKNOWN"], np.nan)

In [57]:
def standardized_data(df):
    df["Quantity"]=pd.to_numeric(df["Quantity"])
    df["Price Per Unit"] = pd.to_numeric( df["Price Per Unit"])
    df["Total Spent"] = pd.to_numeric( df["Total Spent"])
    df["Transaction Date"] = pd.to_datetime(df["Transaction Date"])
    return df
standardized_data(df).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    10000 non-null  object        
 1   Item              9031 non-null   object        
 2   Quantity          9521 non-null   float64       
 3   Price Per Unit    9467 non-null   float64       
 4   Total Spent       9498 non-null   float64       
 5   Payment Method    6822 non-null   object        
 6   Location          6039 non-null   object        
 7   Transaction Date  9540 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 625.1+ KB


# 3. Handling missing Values

In [59]:
from sklearn.impute import SimpleImputer
def handle_missing_values(df):
    numerical_columns = df.select_dtypes(include=["float64","int64"]).columns
    if len(numerical_columns) > 0:
        numeric_imputer = SimpleImputer(strategy="median")
        df[numerical_columns] = numeric_imputer.fit_transform(df[numerical_columns])
        
        categorical_columns = df.select_dtypes(include=["object"]).columns
        if len(categorical_columns) > 0:
            categorical_imputer = SimpleImputer(strategy="most_frequent")
            df[categorical_columns] = categorical_imputer.fit_transform(df[categorical_columns])
        df['Transaction Date'] = df['Transaction Date'].interpolate()       
     
    return df        
    
handle_missing_values(df).info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    10000 non-null  object        
 1   Item              10000 non-null  object        
 2   Quantity          10000 non-null  float64       
 3   Price Per Unit    10000 non-null  float64       
 4   Total Spent       10000 non-null  float64       
 5   Payment Method    10000 non-null  object        
 6   Location          10000 non-null  object        
 7   Transaction Date  10000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 625.1+ KB


# 4. Changing Numeric data to int64

In [61]:
df["Quantity"] = df["Quantity"].astype("int64")
df["Price Per Unit"] = df["Price Per Unit"].astype("int64")
df["Total Spent"] = df["Total Spent"].astype("int64")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    10000 non-null  object        
 1   Item              10000 non-null  object        
 2   Quantity          10000 non-null  int64         
 3   Price Per Unit    10000 non-null  int64         
 4   Total Spent       10000 non-null  int64         
 5   Payment Method    10000 non-null  object        
 6   Location          10000 non-null  object        
 7   Transaction Date  10000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 625.1+ KB


# 5. Detecting and handling Outlier 

In [63]:
def remove_outliers(df):
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    outliers_removed = {}
    
    for column in numeric_columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count outliers before removing
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)].shape[0]
        
        # Cap the values instead of removing them
        df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
        
        if outliers > 0:
            outliers_removed[column] = outliers
            
    return df, outliers_removed

df_1 = df.copy()
df_1, outliers_removed = remove_outliers(df_1)

# 5. Validating the results

In [66]:
# Sort by date
df_1 = df_1.sort_values(by='Transaction Date')
# Remove time
df_1['Transaction Date'] = df_1['Transaction Date'].dt.date
    

In [68]:
def validate_cleaning(df, original_shape, cleaning_report):
    validation_results = {
        'rows_remaining': len(df),
        'missing_values_remaining': df.isnull().sum().sum(),
        'duplicates_remaining': df.duplicated().sum(),
        'data_loss_percentage': (1 - len(df)/original_shape[0]) * 100
    }
    
    cleaning_report['validation'] = validation_results
    return cleaning_report

In [70]:
cleaning_report = {}
report = validate_cleaning(df_1, df_1.shape,cleaning_report)
report

{'validation': {'rows_remaining': 10000,
  'missing_values_remaining': 0,
  'duplicates_remaining': 0,
  'data_loss_percentage': 0.0}}

In [72]:
df_1.to_csv("Cleaned Cafe Sales_final1.csv")